In [1]:
import pandas as pd
pd.set_option('display.max_columns', 30)
import numpy as np
import random
import joblib

### Reading Data

In [2]:
df = pd.read_csv('heart_2020_cleaned.csv')

In [3]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

In [5]:
df.isnull().sum()

HeartDisease        0
BMI                 0
Smoking             0
AlcoholDrinking     0
Stroke              0
PhysicalHealth      0
MentalHealth        0
DiffWalking         0
Sex                 0
AgeCategory         0
Race                0
Diabetic            0
PhysicalActivity    0
GenHealth           0
SleepTime           0
Asthma              0
KidneyDisease       0
SkinCancer          0
dtype: int64

In [6]:
for col in df.columns:
    print(f'{col}: {df[col].unique()}')
    print()

HeartDisease: ['No' 'Yes']

BMI: [16.6  20.34 26.58 ... 62.42 51.46 46.56]

Smoking: ['Yes' 'No']

AlcoholDrinking: ['No' 'Yes']

Stroke: ['No' 'Yes']

PhysicalHealth: [ 3.  0. 20. 28.  6. 15.  5. 30.  7.  1.  2. 21.  4. 10. 14. 18.  8. 25.
 16. 29. 27. 17. 24. 12. 23. 26. 22. 19.  9. 13. 11.]

MentalHealth: [30.  0.  2.  5. 15.  8.  4.  3. 10. 14. 20.  1.  7. 24.  9. 28. 16. 12.
  6. 25. 17. 18. 21. 29. 22. 13. 23. 27. 26. 11. 19.]

DiffWalking: ['No' 'Yes']

Sex: ['Female' 'Male']

AgeCategory: ['55-59' '80 or older' '65-69' '75-79' '40-44' '70-74' '60-64' '50-54'
 '45-49' '18-24' '35-39' '30-34' '25-29']

Race: ['White' 'Black' 'Asian' 'American Indian/Alaskan Native' 'Other'
 'Hispanic']

Diabetic: ['Yes' 'No' 'No, borderline diabetes' 'Yes (during pregnancy)']

PhysicalActivity: ['Yes' 'No']

GenHealth: ['Very good' 'Fair' 'Good' 'Poor' 'Excellent']

SleepTime: [ 5.  7.  8.  6. 12.  4.  9. 10. 15.  3.  2.  1. 16. 18. 14. 20. 11. 13.
 17. 24. 19. 21. 22. 23.]

Asthma: ['Yes' 'No']


### Data Transformation

In [7]:
len(df['AgeCategory'])

319795

In [8]:
len(df['GenHealth'])

319795

In [9]:
gen_health_conversion = {"Excellent":5, 'Very good': 4, 'Good': 3, 'Fair': 2, 'Poor': 1}
gen_health_conversion

{'Excellent': 5, 'Very good': 4, 'Good': 3, 'Fair': 2, 'Poor': 1}

In [10]:
gen_health_conversion = {"Excellent": 5, 'Very good': 4, 'Good': 3, 'Fair': 2, 'Poor': 1}

def convert_age_and_gen_health(row):
    '''
    Converts AgeCategory string into random integer value between the given range, and Encodes GenHealth Data as well.
    Encoding used for GenHealth is 'Excellent': 5, 'Very good': 4, 'Good': 3, 'Fair': 2, 'Poor': 1
    '''
    age_range = row['AgeCategory']
    if age_range != '80 or older':
        min_age = int(age_range[0:2])
        max_age = int(age_range[3:])
    else:
        min_age = 80
        max_age = 95
    age = np.random.randint(min_age, max_age + 1)
    row['AgeCategory'] = age
    row['GenHealth'] = gen_health_conversion.get(row['GenHealth'], row['GenHealth'])
    
    
    return row

In [11]:
def transform_data(df):
    '''
    Applies data transformation techniques to the given DataFrame to convert all the Categorical Data into Nominal Data
    '''
    # Converting Age and genHealth
    df = df.apply(convert_age_and_gen_health, axis=1)
    
    # Label Encoding
    df['HeartDisease'] = df['HeartDisease'].replace(['No', 'Yes'], [0, 1])
    
    # Feature Encoding
    df['Smoking'] = df['Smoking'].replace(['No', 'Yes'], [0, 1])
    df['AlcoholDrinking'] = df['AlcoholDrinking'].replace(['No', 'Yes'], [0, 1])
    df['Stroke'] = df['Stroke'].replace(['No', 'Yes'], [0, 1])
    df['Asthma'] = df['Asthma'].replace(['No', 'Yes'], [0, 1])
    df['DiffWalking'] = df['DiffWalking'].replace(['No', 'Yes'], [0, 1])
    df['KidneyDisease'] = df['KidneyDisease'].replace(['No', 'Yes'], [0, 1])
    df['SkinCancer'] = df['SkinCancer'].replace(['No', 'Yes'], [0, 1])
    df['PhysicalActivity'] = df['PhysicalActivity'].replace(['No', 'Yes'], [0, 1])
    df['Sex'] = df['Sex'].replace(['Male', 'Female'], [0, 1])
    df = pd.get_dummies(df, columns=['Diabetic'], prefix='Diabetic')
    df = pd.get_dummies(df, columns=['Race'], prefix='Race')
    
    return df

In [12]:
df = transform_data(df)

In [13]:
df.rename(columns={"AgeCategory":"Age"}, inplace=True)

In [14]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Age,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,Diabetic_No,"Diabetic_No, borderline diabetes",Diabetic_Yes,Diabetic_Yes (during pregnancy),Race_American Indian/Alaskan Native,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White
0,0,16.60,1,0,0,3.0,30.0,0,1,58,1,4,5.0,1,0,1,0,0,1,0,0,0,0,0,0,1
1,0,20.34,0,0,1,0.0,0.0,0,1,94,1,4,7.0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,0,26.58,1,0,0,20.0,30.0,0,0,69,1,2,8.0,1,0,0,0,0,1,0,0,0,0,0,0,1
3,0,24.21,0,0,0,0.0,0.0,0,1,76,0,3,6.0,0,0,1,1,0,0,0,0,0,0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,1,43,1,4,8.0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [15]:
df.dtypes

HeartDisease                             int64
BMI                                    float64
Smoking                                  int64
AlcoholDrinking                          int64
Stroke                                   int64
PhysicalHealth                         float64
MentalHealth                           float64
DiffWalking                              int64
Sex                                      int64
Age                                      int64
PhysicalActivity                         int64
GenHealth                                int64
SleepTime                              float64
Asthma                                   int64
KidneyDisease                            int64
SkinCancer                               int64
Diabetic_No                              uint8
Diabetic_No, borderline diabetes         uint8
Diabetic_Yes                             uint8
Diabetic_Yes (during pregnancy)          uint8
Race_American Indian/Alaskan Native      uint8
Race_Asian   

### Data Normalization

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 26 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   HeartDisease                         319795 non-null  int64  
 1   BMI                                  319795 non-null  float64
 2   Smoking                              319795 non-null  int64  
 3   AlcoholDrinking                      319795 non-null  int64  
 4   Stroke                               319795 non-null  int64  
 5   PhysicalHealth                       319795 non-null  float64
 6   MentalHealth                         319795 non-null  float64
 7   DiffWalking                          319795 non-null  int64  
 8   Sex                                  319795 non-null  int64  
 9   Age                                  319795 non-null  int64  
 10  PhysicalActivity                     319795 non-null  int64  
 11  GenHealth    

In [17]:
df.describe()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Age,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,Diabetic_No,"Diabetic_No, borderline diabetes",Diabetic_Yes,Diabetic_Yes (during pregnancy),Race_American Indian/Alaskan Native,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White
count,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.00000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000
mean,0.085595,28.325399,0.412477,0.068097,0.037740,3.37171,3.898366,0.138870,0.524727,54.920934,0.775362,3.595028,7.097075,0.134061,0.036833,0.093244,0.843206,0.021204,0.127588,0.008002,0.016267,0.025229,0.071730,0.085824,0.034172,0.766779
std,0.279766,6.356100,0.492281,0.251912,0.190567,7.95085,7.955235,0.345812,0.499389,18.722662,0.417344,1.042918,1.436007,0.340718,0.188352,0.290775,0.363607,0.144065,0.333631,0.089095,0.126499,0.156819,0.258041,0.280104,0.181671,0.422883
min,0.000000,12.020000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,18.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,24.030000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,40.000000,1.000000,3.000000,6.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,27.340000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,57.000000,1.000000,4.000000,7.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,31.420000,1.000000,0.000000,0.000000,2.00000,3.000000,0.000000,1.000000,69.000000,1.000000,4.000000,8.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,94.850000,1.000000,1.000000,1.000000,30.00000,30.000000,1.000000,1.000000,95.000000,1.000000,5.000000,24.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
sc = StandardScaler()

In [26]:
df[df.columns[1:]]

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Age,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,Diabetic_No,"Diabetic_No, borderline diabetes",Diabetic_Yes,Diabetic_Yes (during pregnancy),Race_American Indian/Alaskan Native,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White
0,16.60,1,0,0,3.0,30.0,0,1,58,1,4,5.0,1,0,1,0,0,1,0,0,0,0,0,0,1
1,20.34,0,0,1,0.0,0.0,0,1,94,1,4,7.0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,26.58,1,0,0,20.0,30.0,0,0,69,1,2,8.0,1,0,0,0,0,1,0,0,0,0,0,0,1
3,24.21,0,0,0,0.0,0.0,0,1,76,0,3,6.0,0,0,1,1,0,0,0,0,0,0,0,0,1
4,23.71,0,0,0,28.0,0.0,1,1,43,1,4,8.0,0,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,27.41,1,0,0,7.0,0.0,1,0,62,0,2,6.0,1,0,0,0,0,1,0,0,0,0,1,0,0
319791,29.84,1,0,0,0.0,0.0,0,0,35,1,4,5.0,1,0,0,1,0,0,0,0,0,0,1,0,0
319792,24.24,0,0,0,0.0,0.0,0,1,49,1,3,6.0,0,0,0,1,0,0,0,0,0,0,1,0,0
319793,32.81,0,0,0,0.0,0.0,0,1,25,0,3,12.0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [27]:
columns = df.columns[1:]
columns

Index(['BMI', 'Smoking', 'AlcoholDrinking', 'Stroke', 'PhysicalHealth',
       'MentalHealth', 'DiffWalking', 'Sex', 'Age', 'PhysicalActivity',
       'GenHealth', 'SleepTime', 'Asthma', 'KidneyDisease', 'SkinCancer',
       'Diabetic_No', 'Diabetic_No, borderline diabetes', 'Diabetic_Yes',
       'Diabetic_Yes (during pregnancy)',
       'Race_American Indian/Alaskan Native', 'Race_Asian', 'Race_Black',
       'Race_Hispanic', 'Race_Other', 'Race_White'],
      dtype='object')

In [24]:
sc.fit(df[df.columns[1:]])

StandardScaler()

In [28]:
data = sc.transform(df[df.columns[1:]])

In [29]:
data

array([[-1.84475016,  1.19347355, -0.27031975, ..., -0.30639997,
        -0.18809818,  0.55150451],
       [-1.25633812, -0.83789038, -0.27031975, ..., -0.30639997,
        -0.18809818,  0.55150451],
       [-0.27460254,  1.19347355, -0.27031975, ..., -0.30639997,
        -0.18809818,  0.55150451],
       ...,
       [-0.64275338, -0.83789038, -0.27031975, ...,  3.26370786,
        -0.18809818, -1.81322182],
       [ 0.70555975, -0.83789038, -0.27031975, ...,  3.26370786,
        -0.18809818, -1.81322182],
       [ 2.86883929, -0.83789038, -0.27031975, ...,  3.26370786,
        -0.18809818, -1.81322182]])

In [30]:
df[df.columns[1:]] = data

In [33]:
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Age,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,Diabetic_No,"Diabetic_No, borderline diabetes",Diabetic_Yes,Diabetic_Yes (during pregnancy),Race_American Indian/Alaskan Native,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White
0,0,-1.844750,1.193474,-0.27032,-0.198040,-0.046751,3.281069,-0.401578,0.951711,0.164457,0.538256,0.388307,-1.460354,2.541515,-0.195554,3.118419,-2.319006,-0.147185,2.614905,-0.089814,-0.128591,-0.160878,-0.27798,-0.306400,-0.188098,0.551505
1,0,-1.256338,-0.837890,-0.27032,5.049478,-0.424070,-0.490039,-0.401578,0.951711,2.087263,0.538256,0.388307,-0.067601,-0.393466,-0.195554,-0.320675,0.431219,-0.147185,-0.382423,-0.089814,-0.128591,-0.160878,-0.27798,-0.306400,-0.188098,0.551505
2,0,-0.274603,1.193474,-0.27032,-0.198040,2.091388,3.281069,-0.401578,-1.050739,0.751981,0.538256,-1.529392,0.628776,2.541515,-0.195554,-0.320675,-2.319006,-0.147185,2.614905,-0.089814,-0.128591,-0.160878,-0.27798,-0.306400,-0.188098,0.551505
3,0,-0.647473,-0.837890,-0.27032,-0.198040,-0.424070,-0.490039,-0.401578,0.951711,1.125860,-1.857852,-0.570542,-0.763977,-0.393466,-0.195554,3.118419,0.431219,-0.147185,-0.382423,-0.089814,-0.128591,-0.160878,-0.27798,-0.306400,-0.188098,0.551505
4,0,-0.726138,-0.837890,-0.27032,-0.198040,3.097572,-0.490039,2.490174,0.951711,-0.636712,0.538256,0.388307,0.628776,-0.393466,-0.195554,-0.320675,0.431219,-0.147185,-0.382423,-0.089814,-0.128591,-0.160878,-0.27798,-0.306400,-0.188098,0.551505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,1,-0.144019,1.193474,-0.27032,-0.198040,0.456341,-0.490039,2.490174,-1.050739,0.378102,-1.857852,-1.529392,-0.763977,2.541515,-0.195554,-0.320675,-2.319006,-0.147185,2.614905,-0.089814,-0.128591,-0.160878,-0.27798,3.263708,-0.188098,-1.813222
319791,0,0.238291,1.193474,-0.27032,-0.198040,-0.424070,-0.490039,-0.401578,-1.050739,-1.064003,0.538256,0.388307,-1.460354,2.541515,-0.195554,-0.320675,0.431219,-0.147185,-0.382423,-0.089814,-0.128591,-0.160878,-0.27798,3.263708,-0.188098,-1.813222
319792,0,-0.642753,-0.837890,-0.27032,-0.198040,-0.424070,-0.490039,-0.401578,0.951711,-0.316245,0.538256,-0.570542,-0.763977,-0.393466,-0.195554,-0.320675,0.431219,-0.147185,-0.382423,-0.089814,-0.128591,-0.160878,-0.27798,3.263708,-0.188098,-1.813222
319793,0,0.705560,-0.837890,-0.27032,-0.198040,-0.424070,-0.490039,-0.401578,0.951711,-1.598116,-1.857852,-0.570542,3.414282,-0.393466,-0.195554,-0.320675,0.431219,-0.147185,-0.382423,-0.089814,-0.128591,-0.160878,-0.27798,3.263708,-0.188098,-1.813222


In [18]:
# Max values of each column used for normalization
max_vector = df.max()

In [19]:
max_vector = max_vector.to_numpy()

In [20]:
max_vector

array([ 1.  , 94.85,  1.  ,  1.  ,  1.  , 30.  , 30.  ,  1.  ,  1.  ,
       95.  ,  1.  ,  5.  , 24.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,
        1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ])

In [21]:
def apply_normalization(row):
    '''
    Normalizes the given row so that all the features are between 0-1
    '''
    row = row / max_vector
    return row

In [22]:
df = df.apply(apply_normalization, axis = 1)

In [34]:
df.describe()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Age,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,Diabetic_No,"Diabetic_No, borderline diabetes",Diabetic_Yes,Diabetic_Yes (during pregnancy),Race_American Indian/Alaskan Native,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White
count,319795.000000,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05,3.197950e+05
mean,0.085595,7.247738e-16,3.581654e-17,6.023488e-17,-1.905253e-17,5.814632e-17,-4.332645e-17,-4.122679e-17,-1.328678e-17,-1.550865e-16,-8.212029e-17,2.406285e-17,1.727170e-16,-3.899936e-17,1.944136e-17,-9.918425e-17,1.643961e-16,-1.537811e-17,1.639740e-17,3.708300e-17,2.244088e-18,-8.000952e-17,-3.163942e-17,-8.923028e-17,-3.866053e-18,9.136327e-17
std,0.279766,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,0.000000,-2.565319e+00,-8.378904e-01,-2.703198e-01,-1.980403e-01,-4.240698e-01,-4.900386e-01,-4.015783e-01,-1.050739e+00,-1.971995e+00,-1.857852e+00,-2.488242e+00,-4.245859e+00,-3.934661e-01,-1.955544e-01,-3.206753e-01,-2.319006e+00,-1.471855e-01,-3.824231e-01,-8.981397e-02,-1.285910e-01,-1.608777e-01,-2.779805e-01,-3.064000e-01,-1.880982e-01,-1.813222e+00
25%,0.000000,-6.757926e-01,-8.378904e-01,-2.703198e-01,-1.980403e-01,-4.240698e-01,-4.900386e-01,-4.015783e-01,-1.050739e+00,-7.969464e-01,5.382561e-01,-5.705424e-01,-7.639770e-01,-3.934661e-01,-1.955544e-01,-3.206753e-01,4.312193e-01,-1.471855e-01,-3.824231e-01,-8.981397e-02,-1.285910e-01,-1.608777e-01,-2.779805e-01,-3.064000e-01,-1.880982e-01,5.515045e-01
50%,0.000000,-1.550322e-01,-8.378904e-01,-2.703198e-01,-1.980403e-01,-4.240698e-01,-4.900386e-01,-4.015783e-01,9.517109e-01,1.110456e-01,5.382561e-01,3.883072e-01,-6.760053e-02,-3.934661e-01,-1.955544e-01,-3.206753e-01,4.312193e-01,-1.471855e-01,-3.824231e-01,-8.981397e-02,-1.285910e-01,-1.608777e-01,-2.779805e-01,-3.064000e-01,-1.880982e-01,5.515045e-01
75%,0.000000,4.868719e-01,1.193474e+00,-2.703198e-01,-1.980403e-01,-1.725240e-01,-1.129278e-01,-4.015783e-01,9.517109e-01,7.519811e-01,5.382561e-01,3.883072e-01,6.287760e-01,-3.934661e-01,-1.955544e-01,-3.206753e-01,4.312193e-01,-1.471855e-01,-3.824231e-01,-8.981397e-02,-1.285910e-01,-1.608777e-01,-2.779805e-01,-3.064000e-01,-1.880982e-01,5.515045e-01
max,1.000000,1.046628e+01,1.193474e+00,3.699323e+00,5.049478e+00,3.349118e+00,3.281069e+00,2.490174e+00,9.517109e-01,2.140675e+00,5.382561e-01,1.347157e+00,1.177080e+01,2.541515e+00,5.113667e+00,3.118419e+00,4.312193e-01,6.794148e+00,2.614905e+00,1.113412e+01,7.776593e+00,6.215904e+00,3.597375e+00,3.263708e+00,5.316373e+00,5.515045e-01


In [35]:
df.to_csv('Processed_Heart_Data.csv')

In [36]:
joblib.dump(sc, 'scaler.model')

['scaler.model']